In [4]:
import pandas as pd
import mysql.connector

In [5]:
connection = mysql.connector.connect(
    host= 'localhost',         
    user= 'root',             
    password= '123456', 
    database = 'day_50'     
)
cursor = connection.cursor()

In [3]:
cursor.execute("""
CREATE TABLE customers (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL, 
    city VARCHAR(200) NOT NULL
);
""")

In [4]:
cursor.execute("""
INSERT INTO customers (id, name, city) VALUES
(1, 'Alice', 'Hanoi'),
(2, 'Bob', 'Ho Chi Minh'),
(3, 'Charlie', 'Hanoi');
""")

connection.commit()

In [5]:
cursor.execute("""
CREATE TABLE orders (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    total INTEGER NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers(id)
);
""")

In [6]:
cursor.execute("""
INSERT INTO orders (id, customer_id, order_date, total) VALUES
(1, 1, '2024-01-10', 500),
(2, 1, '2024-03-01', 800),
(3, 2, '2024-02-20', 1200),
(4, 3, '2024-01-15', 200);
""")

connection.commit()

In [7]:
cursor.execute("""
CREATE TABLE products (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    price INTEGER NOT NULL
);
""")

In [8]:
cursor.execute("""
INSERT INTO products (id, name, price) VALUES
(1, 'Laptop', 1500),
(2, 'Mouse', 50),
(3, 'Keyboard', 100),
(4, 'Monitor', 300);
""")

connection.commit()

In [9]:
cursor.execute("""
CREATE TABLE order_items (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (product_id) REFERENCES products(id)
);
""")

In [10]:
cursor.execute("""
INSERT INTO order_items (id, order_id, product_id, quantity) VALUES
(1, 1, 2, 2),
(2, 1, 3, 1),
(3, 2, 1, 1),
(4, 2, 4, 1),
(5, 3, 1, 1);
""")

connection.commit()

In [11]:
cursor.execute("""
CREATE TABLE employees (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(200) NOT NULL,
    department VARCHAR(200) NOT NULL
);
""")

In [12]:
cursor.execute("""
INSERT INTO employees (id, name, department) VALUES
(1, 'David', 'Sales'),
(2, 'Emma', 'Support'),
(3, 'Frank', 'Sales');
""")

connection.commit()

In [13]:
cursor.execute("""
CREATE TABLE order_assignments (
    id INTEGER PRIMARY KEY AUTO_INCREMENT,
    order_id INTEGER NOT NULL,
    employee_id INTEGER NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(id),
    FOREIGN KEY (employee_id) REFERENCES employees(id)
);
""")

In [14]:
cursor.execute("""
INSERT INTO order_assignments (id, order_id, employee_id) VALUES
(1, 1, 1),
(2, 2, 3),
(3, 3, 2);
""")

connection.commit()

### Bài 1

In [6]:
cursor.execute("""
CREATE TABLE order_logs(
    id INT AUTO_INCREMENT PRIMARY KEY,
    log_message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

In [9]:
cursor.execute("""
CREATE TRIGGER OrderDeleteLogger
BEFORE DELETE ON orders
FOR EACH ROW
BEGIN
    INSERT INTO order_logs (log_message)
    VALUES(CONCAT('Delete order ID: ', OLD.id , ' - total: ', OLD.total));
END
""")
connection.commit()

In [12]:
cursor.execute("DELETE FROM order_items WHERE order_id = 1")
cursor.execute("DELETE FROM order_assignments WHERE order_id = 1")
cursor.execute("DELETE FROM orders WHERE id = 1")
connection.commit()

cursor.execute("SELECT * FROM order_logs")
for row in cursor.fetchall():
    print(row)

(1, 'Delete order ID: 4 - total: 200', datetime.datetime(2025, 4, 21, 12, 42, 47))
(3, 'Delete order ID: 1 - total: 500', datetime.datetime(2025, 4, 21, 12, 43, 49))


### Bài 2

In [14]:
cursor.execute("""
CREATE TRIGGER UpdateOrderTotalOnInsert
AFTER INSERT ON order_items
FOR EACH ROW
BEGIN
    DECLARE item_price INT;
               
    SELECT price INTO item_price 
    FROM products 
    WHERE id = NEW.product_id;
               
    UPDATE orders 
    SET total = total + (item_price * NEW.quantity) 
    WHERE id = NEW.order_id;
END
""")
connection.commit()

In [15]:
cursor.execute("INSERT INTO order_items (order_id, product_id, quantity) VALUES (2, 3, 2)")
connection.commit()

cursor.execute("SELECT id, total FROM orders WHERE id = 2")
print(cursor.fetchone())

(2, 1000)


### Bài 3

In [16]:
cursor.execute("""
CREATE TRIGGER PreventHighPriceUpdate
BEFORE UPDATE ON products
FOR EACH ROW
BEGIN
    IF NEW.price > 1000 THEN
        SIGNAL SQLSTATE '45000' 
        SET MESSAGE_TEXT = 'Cannot update product price to a value greater than 1000';
    END IF;
END
""")
connection.commit()

In [17]:
try:
    cursor.execute("UPDATE products SET price = 900 WHERE id = 2")
    connection.commit()
    print("Update successful with price = 900")
except Exception as e:
    print(f"Error: {e}")

Update successful with price = 900


In [18]:
try:
    cursor.execute("UPDATE products SET price = 1200 WHERE id = 2")
    connection.commit()
    print("Update successful with price = 1200")
except Exception as e:
    print(f"Error: {e}")

Error: 1644 (45000): Cannot update product price to a value greater than 1000


In [19]:
cursor.execute("SELECT id, name, price FROM products WHERE id = 2")
print(cursor.fetchone())

(2, 'Mouse', 900)


### Bài 4

In [20]:
cursor.execute("""
CREATE TABLE customers_deleted(
    id INT PRIMARY KEY,
    name TEXT,
    city TEXT,
    deleted_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

In [21]:
cursor.execute("""
CREATE TRIGGER CustomerDeleteLogger
BEFORE DELETE ON customers
FOR EACH ROW
BEGIN
    INSERT INTO customers_deleted (id, name, city, deleted_at)
    VALUES(OLD.id, OLD.name, OLD.city, NOW());
END
""")

In [25]:
cursor.execute('DELETE FROM order_items WHERE order_id IN (SELECT id FROM orders WHERE customer_id = 1);')
cursor.execute('DELETE FROM order_assignments WHERE order_id IN (SELECT id FROM orders WHERE customer_id = 1);')
cursor.execute('DELETE FROM orders WHERE customer_id = 1;')
cursor.execute('DELETE FROM customers WHERE id = 1;')
connection.commit()

cursor.execute("SELECT * FROM customers_deleted")
for row in cursor.fetchall():
    print(row)

(1, 'Alice', 'Hanoi', datetime.datetime(2025, 4, 21, 13, 19, 52))


In [26]:
cursor.execute("SELECT * FROM customers WHERE id = 1")
print(cursor.fetchone())

None


### Bài 5

In [34]:
cursor.execute("""
CREATE TABLE order_alerts(
    id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    alert_message TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
""")

In [36]:
cursor.execute("""
CREATE TRIGGER OrderAlertTrigger
AFTER INSERT ON orders
FOR EACH ROW
BEGIN
    DECLARE customer_city VARCHAR(200);
    SELECT city INTO customer_city
     FROM customers 
    WHERE id = NEW.customer_id;
               
    IF NEW.total > 1000 AND customer_city = 'Hanoi' THEN
        INSERT INTO order_alerts (order_id, alert_message)
        VALUES(NEW.id, CONCAT('Order total from Hanoi is too high: ', NEW.total));
    END IF;
END
""")

In [37]:
cursor.execute("INSERT INTO customers (name, city) VALUES ('Thang', 'Hanoi')")
connection.commit()

In [38]:
cursor.execute("SELECT id FROM customers ORDER BY id DESC LIMIT 1")
new_customer_id = cursor.fetchone()[0]

cursor.execute(f"INSERT INTO orders (customer_id, order_date, total) VALUES ({new_customer_id}, '2024-04-21', 1500)")
connection.commit()

In [39]:
cursor.execute("SELECT * FROM order_alerts")
alerts = cursor.fetchall()
for alert in alerts:
    print(alert)

(1, 5, 'Order total from Hanoi is too high: 1500', datetime.datetime(2025, 4, 21, 13, 36, 33))
